### UP 2010 Clean up

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
up_2010 = pd.read_csv("up_gp_sarpanch_2010.csv", low_memory = False)
up_2010.head()

,original_filename,page,order,district_name,sr_no,gp_code,block_code,block_name,sr_gp_combo,gp_name,gp_reservation_status,elected_sarpanch_name,husband_spouse_name,candidate_res_status,age,sex,education,address
0,gpp_1901.pdf,20,55362,संत रविदास नगर,475,40,6,अभोली,गंगारामपुर,NaN,अनुसूचित जाति - महिला,बेबी,सुनील,अनुसूचित जाति महिला -,NaN,30 महिला,साक्षर,"गंगारामपुर , गौरा"
1,gpp_1901.pdf,14,55211,संत रविदास नगर,338,NaN,5,भदोही,338 6 सरबतखानी,NaN,अनुसूचित जाति - महिला,संतरादेवी,सुनीलकुमार,अनुसूचित जाति - महिला,NaN,35 महिला,साक्षर,सरबतखानी
2,gpp_1901.pdf,14,55219,संत रविदास नगर,346,NaN,5,भदोही,346 14 डोमनपुर गिरिया,NaN,अनुसूचित जाति - महिला,सुरसत्तीदेवी,भोलानाथ,अनुसूचित जाति महिला -,NaN,38 महिला,साक्षर,डोमनपुर गिरिया
3,gpp_1901.pdf,14,55215,संत रविदास नगर,342,NaN,5,भदोही,342 10 दरीबपुर,NaN,अनुसूचित जाति महिला,शान्ती,मोहनलाल,अनुसूचित जाति महिला -,NaN,45 महिला,साक्षर,दरीबपुर
4,gpp_1901.pdf,19,55359,संत रविदास नगर,474,39,6,अभोली,सराय होला,NaN,अनुसूचित जाति महिला,धर्मादेवी,बंशीधर,अनुसूचित जाति महिला -,NaN,48 महिला साक्षर,NaN,"सरायहोला , अभोली"


### Let's get GP Name sorted

In [3]:
up_2010['gp_name_temp'] = up_2010['sr_gp_combo'].str.strip().str.replace('[0-9\[\]]', '', regex=True)

In [4]:
### Normalize GP Name

up_2010['gp_name'] = up_2010['gp_name'].str.strip()\
                        .str.replace(r'^(\s?-\s?|\s?--\s?)|(\s-|\s--|\s---)$', '', regex=True)\
                        .str.replace(r'\s{2,}', ' ', regex=True)\
                        .str.strip()

# Let's take out reservation status from GP Name
gp_res_status = [
    'अनारक्षित',
    'महिला',
    'अनुसूचित जाति - महिला',
    'पिछड़ी जाति - महिला',
    'पिछड़ी जाति',
    'अनुसूचित जाति',
    'पिछड़ी जाति महिला -',
    'अनुसूचित जाति महिला -'
    'महिला'
    'अनुसूचित जाति महिला -',
    'पिछड़ी जाति महिला',
    'अनुसूचित जाति महिला',
    'पिछड़ी जाति महिला -'
    'अनुसूचित जाति महिला -',
    'पिछड़ी जाति महिला --',
    'अनुसूचित जाति महिला - '
]
up_2010['gp_res_status_temp'] = np.where(up_2010['gp_name'].str.strip().isin(gp_res_status), up_2010['gp_name'], '')
up_2010['gp_name'] = np.where(~up_2010['gp_name'].isin(gp_res_status), up_2010['gp_name'], '')
up_2010['gp_name'].value_counts()[:20]

              159
रामपुर         52
रसूलपुर        50
गोपालपुर       49
रामनगर         42
रायपुर         41
शाहपुर         41
सलेमपुर        39
सिकन्दरपुर     36
गौरा           36
जगदीशपुर       36
भगवानपुर       35
जलालपुर        35
नरायनपुर       34
दौलतपुर        31
बहादुरपुर      31
कल्यानपुर      31
बरौली          30
सुल्तानपुर     29
खानपुर         28
Name: gp_name, dtype: int64

In [5]:
up_2010['gp_name_fin'] = up_2010.apply(lambda x: x['gp_name_temp'] if x['gp_name_temp'] != '' 
                            else x['gp_name'], axis=1)
up_2010['gp_name_fin']

0                    गंगारामपुर
1                      सरबतखानी
2                डोमनपुर गिरिया
3                       दरीबपुर
4                     सराय होला
                  ...          
51856       देउरी अनुसूचित जाति
51857       धमौरा अनुसूचित जाति
51858    निखोरिया अनुसूचित जाति
51859             वनकटा द्वितीय
51860            बंजरहा बुजुर्ग
Name: gp_name_fin, Length: 51861, dtype: object

In [6]:
up_2010['gp_reservation_status'].value_counts()[:20]

अनारक्षित                17907
पिछड़ी जाति               9467
महिला                     8231
अनुसूचित जाति             7155
पिछड़ी जाति - महिला       2445
अनुसूचित जाति - महिला     2104
पिछड़ी जाति महिला -       1380
अनुसूचित जाति महिला -     1339
पिछड़ी जाति महिला          585
अनुसूचित जाति महिला        348
पिछड़ी जाति महिला -         187
पिछड़ी जाति महिला --       111
पिछडी जाति                  64
पिछडी जाति महिला -          56
पिछडी जाति - महिला          53
पिछड़ी जाति महिला            37
- पिछड़ी जाति महिला         29
- अनुसूचित जाति महिला       20
अनुसूचित जन जाति            17
पिछड़ी जाति महिला ---       15
Name: gp_reservation_status, dtype: int64

In [7]:
up_2010['gp_res_status_fin'] = up_2010.apply(lambda x: x['gp_res_status_temp'] if x['gp_res_status_temp'] != '' 
                            else x['gp_reservation_status'], axis=1)
up_2010['gp_res_status_fin'].value_counts()[:24]

अनारक्षित                   17906
पिछड़ी जाति                  9467
महिला                        8390
अनुसूचित जाति                7155
पिछड़ी जाति - महिला          2445
अनुसूचित जाति - महिला        2104
पिछड़ी जाति महिला -          1380
अनुसूचित जाति महिला -        1339
पिछड़ी जाति महिला             585
अनुसूचित जाति महिला           348
पिछड़ी जाति महिला -            187
पिछड़ी जाति महिला --          111
पिछडी जाति                     64
पिछडी जाति महिला -             56
पिछडी जाति - महिला             53
पिछड़ी जाति महिला               37
- पिछड़ी जाति महिला            29
- अनुसूचित जाति महिला          20
अनुसूचित जन जाति               17
पिछड़ी जाति महिला ---          15
अनुसूचित जाति महिला --         11
अनुसूचित जन जाति - महिला        8
पिछड़ी जाति - महिला              7
पिछडी जाति महिला                6
Name: gp_res_status_fin, dtype: int64

In [8]:
# Normalize
up_2010['gp_res_status_fin'] = (up_2010['gp_res_status_fin']
                 .str.strip()  # Step 1: Trim leading and trailing spaces   
                 .str.replace(r'^(\s?-\s?|\s?--\s?)|(\s-|\s--|\s---)$', '', regex=True)  # Step 2: Remove specific patterns at the start/end
                 .str.strip()  # Step 1: Trim leading and trailing spaces
                 .str.replace(r'\s{2,}', ' ', regex=True).str.strip())  # Step 3: Replace multiple spaces with a single space
up_2010['gp_res_status_fin'].value_counts()[:20]

अनारक्षित                   17907
पिछड़ी जाति                  9468
महिला                        8390
अनुसूचित जाति                7155
पिछड़ी जाति - महिला          2448
पिछड़ी जाति महिला            2120
अनुसूचित जाति - महिला        2104
अनुसूचित जाति महिला          1719
पिछड़ी जाति महिला              227
पिछडी जाति                     64
पिछडी जाति महिला               63
पिछडी जाति - महिला             53
अनुसूचित जन जाति               17
अनुसूचित जन जाति - महिला        8
पिछड़ी जाति - महिला              7
अनुसूचित जन जाति महिला          4
महिला अनारक्षित                 2
पिछड़ी जाति अनारक्षित           2
अनुसूचित जनजाति महिला           2
-- पिछड़ी जाति महिला            2
Name: gp_res_status_fin, dtype: int64

In [9]:
up_2010['gp_res_status_fin'].value_counts()[:20].index.to_list()

['अनारक्षित',
 'पिछड़ी जाति',
 'महिला',
 'अनुसूचित जाति',
 'पिछड़ी जाति - महिला',
 'पिछड़ी जाति महिला',
 'अनुसूचित जाति - महिला',
 'अनुसूचित जाति महिला',
 'पिछड़ी जाति महिला',
 'पिछडी जाति',
 'पिछडी जाति महिला',
 'पिछडी जाति - महिला',
 'अनुसूचित जन जाति',
 'अनुसूचित जन जाति - महिला',
 'पिछड़ी जाति - महिला',
 'अनुसूचित जन जाति महिला',
 'महिला अनारक्षित',
 'पिछड़ी जाति अनारक्षित',
 'अनुसूचित जनजाति महिला',
 '-- पिछड़ी जाति महिला']

In [10]:
# Define the list of substrings to check for
valid_categories = up_2010['gp_res_status_fin'].value_counts()[:20].index.to_list()

up_2010['gp_res_status_fin'] = np.where(up_2010['gp_res_status_fin'].isin(valid_categories), up_2010['gp_res_status_fin'], '')
up_2010['gp_res_status_fin'].value_counts()[:20]

अनारक्षित                   17907
पिछड़ी जाति                  9468
महिला                        8390
अनुसूचित जाति                7155
पिछड़ी जाति - महिला          2448
पिछड़ी जाति महिला            2120
अनुसूचित जाति - महिला        2104
अनुसूचित जाति महिला          1719
पिछड़ी जाति महिला              227
                               99
पिछडी जाति                     64
पिछडी जाति महिला               63
पिछडी जाति - महिला             53
अनुसूचित जन जाति               17
अनुसूचित जन जाति - महिला        8
पिछड़ी जाति - महिला              7
अनुसूचित जन जाति महिला          4
पिछड़ी जाति अनारक्षित           2
अनुसूचित जनजाति महिला           2
महिला अनारक्षित                 2
Name: gp_res_status_fin, dtype: int64

In [11]:
def process_info(info):
    if not isinstance(info, str):
        return "", "", None  # Handle non-string data directly

    # Initialize default values for gender and age
    gender = ""
    age = None

    keywords = {'महिला': 'woman', 'पुरुष': 'man'}
    age_pattern = re.compile(r'\b\d+\b')  # Matches whole numbers

    # Extract age from the string (assuming the first number found is the age)
    found_age = age_pattern.findall(info)
    if found_age:
        age = int(found_age[0])  # Convert the first found age to integer
        info = age_pattern.sub("", info).strip()  # Remove the age from the string

    # Check for and process keywords
    for keyword, eng_gender in keywords.items():
        if keyword in info:
            gender = keyword #eng_gender
            info = info.replace(keyword, '').strip()  # Strip keyword and any extra spaces

    return info, gender, age

### Split Cols on the Right

In [12]:
up_2010['candidate_res_status_fin'], up_2010['cand_sex_t'], up_2010['age_t'] = zip(*up_2010['candidate_res_status'].apply(process_info))
up_2010['candidate_res_status_fin'].value_counts()

पिछड़ी जाति                        21703
अनारक्षित                          10035
अनुसूचित जाति                       9958
पिछड़ी जाति -                       3420
                                    2687
अनुसूचित जाति  -                    1273
अनुसूचित जाति -                      939
पिछड़ी जाति  -                       741
पिछडी जाति                           340
- अनुसूचित जाति                      241
अनुसूचित जा                          142
अनुसूचित जन जाति                      60
पिछड़ी जाति                            59
पिछड़ी जाति  -                         57
- पिछड़ी जाति                         56
अनुसूचित जात                          40
अनुसूचित जाति  --                     18
अनुसूचित जाि                          17
अनुसूचित जन जाति -                    14
- पिछड़ी जाति                           9
पिछड़ी जाति\nअनारक्षित                 3
-- अनुसूचित जाति                       3
पिछड़ी जाति\nपिछड़ी जाति               2
पिछड़ी जाति\nअनुसूचित जाति             2
अनुसूचित जाति\nप

In [13]:
up_2010['educ_t2'], up_2010['cand_sex_t2'], up_2010['age_t2'] = zip(*up_2010['sex'].apply(process_info))
up_2010['cand_sex_t2'].value_counts()

         37096
पुरुष     8494
महिला     6271
Name: cand_sex_t2, dtype: int64

In [14]:
up_2010['educ_t3'], up_2010['cand_sex_t3'], up_2010['age_t3'] = zip(*up_2010['age'].apply(process_info))
up_2010['cand_sex_t3'].value_counts()

पुरुष    21693
महिला    15346
         14822
Name: cand_sex_t3, dtype: int64

In [15]:
up_2010['educ_t4'], up_2010['cand_sex_t4'], up_2010['age_t4'] = zip(*up_2010['education'].apply(process_info))
up_2010['educ_t4'].value_counts()

                                     20813
साक्षर                                6456
जूनियर हाई स्कूल                      5206
इण्टर                                 4849
हाई स्कूल                             4546
प्राईमरी                              3684
स्नातक                                3459
निरक्षर                               1624
परास्नातक                             1206
इण्टर \nप्राईमरी                         2
जूनियर हाई स्कूल जूनियर हाई स्कूल        2
जूनियर हाई स्कूल \nहाई स्कूल             2
मटेहॅू                                   1
हाई स्कूल ग्राम\n                        1
साक्षर जूनियर हाई स्कूल                  1
साक्षर \nहाई स्कूल                       1
परास्नातक \nहाई स्कूल                    1
जूनियर हाई स्कूल \nप्राईमरी              1
जूनियर हाई स्कूल \nसाक्षर                1
प्राईमरी \nजूनियर हाई स्कूल              1
परास्नातक                                1
इण्टर \nजूनियर हाई स्कूल                 1
हाई स्कूल \nसाक्षर                       1
हाई स्कूल  

### Finalize

In [16]:
# Candidate Sex Final
up_2010['cand_sex_fin'] = up_2010.apply(lambda x: x['cand_sex_t2'] if x['cand_sex_t3'] == '' else x['cand_sex_t3'], axis=1)
up_2010['cand_sex_fin'].value_counts()

पुरुष    30187
महिला    21616
            58
Name: cand_sex_fin, dtype: int64

In [17]:
# Candidate Age Final
up_2010['age_fin'] = up_2010.apply(lambda x: x['age_t2'] if x['age_t3'] == '' else x['age_t3'], axis=1)
up_2010['age_fin'].isna().sum()

191

In [18]:
up_2010['age_fin'].describe()

count    51670.000000
mean        41.193652
std         10.048477
min          3.000000
25%         35.000000
50%         40.000000
75%         48.000000
max        340.000000
Name: age_fin, dtype: float64

In [19]:
up_2010['educ_t3'].value_counts()

                                           31326
साक्षर                                      6994
हाई स्कूल                                   3118
प्राईमरी                                    3016
स्नातक                                      2107
इण्टर                                       2098
जूनियर हाई स्कूल                            1336
निरक्षर                                     1164
परास्नातक                                    665
जूनियर हाई                                     3
निरक्षर\nप्राईमरी                              3
प्राईमरी\nनिरक्षर                              2
जूनियर हाई स्कूल जूनियर हाई स्कूल              2
स्नातक\n  इण्टर                                2
जूनियर हाई स्कूल\n  स्नातक                     1
साक्षर\n निरक्षर                               1
निरक्षर\nनिरक्षर                               1
प्राईमरी ग्राम                                 1
प्राईमरी\n  निरक्षर                            1
साक्षर\nइण्टर                                  1
निरक्षर\nसाक्षर     

In [20]:
# Candidate Education
up_2010['educ_fin'] = up_2010.apply(lambda x: x['educ_t2'] if x['educ_t3'] == '' else x['educ_t3'], axis=1)
up_2010['educ_fin'] = up_2010.apply(lambda x: x['educ_fin'] if x['educ_t4'] == '' else x['educ_t4'], axis=1)
up_2010['educ_fin'].value_counts()

साक्षर                                     13453
हाई स्कूल                                   7668
इण्टर                                       6951
प्राईमरी                                    6748
जूनियर हाई स्कूल                            6545
स्नातक                                      5570
निरक्षर                                     2789
परास्नातक                                   1871
                                             211
जूनियर हाई स्कूल जूनियर हाई स्कूल              4
निरक्षर\nप्राईमरी                              3
जूनियर हाई                                     3
प्राईमरी\nनिरक्षर                              2
स्नातक\n  इण्टर                                2
प्राईमरी\n  जूनियर हाई स्कूल                   2
जूनियर हाई स्कूल \nहाई स्कूल                   2
इण्टर \nप्राईमरी                               2
प्राईमरी ग्राम                                 1
प्राईमरी\n  निरक्षर                            1
हाई स्कूल ग्राम\n                              1
साक्षर\n निरक्षर    

In [21]:
up_2010.columns

Index(['original_filename', 'page', 'order', 'district_name', 'sr_no',
       'gp_code', 'block_code', 'block_name', 'sr_gp_combo', 'gp_name',
       'gp_reservation_status', 'elected_sarpanch_name', 'husband_spouse_name',
       'candidate_res_status', 'age', 'sex', 'education', 'address',
       'gp_name_temp', 'gp_res_status_temp', 'gp_name_fin',
       'gp_res_status_fin', 'candidate_res_status_fin', 'cand_sex_t', 'age_t',
       'educ_t2', 'cand_sex_t2', 'age_t2', 'educ_t3', 'cand_sex_t3', 'age_t3',
       'educ_t4', 'cand_sex_t4', 'age_t4', 'cand_sex_fin', 'age_fin',
       'educ_fin'],
      dtype='object')

In [22]:
up_2010.drop(columns =['gp_name_temp', 'gp_res_status_temp', 
              'cand_sex_t', 'age_t', 
              'educ_t2', 'cand_sex_t2', 'age_t2', 
              'cand_sex_t3', 'age_t3', 'educ_t3',
              'cand_sex_t4', 'age_t4', 'educ_t4'], inplace = True)
up_2010.columns

Index(['original_filename', 'page', 'order', 'district_name', 'sr_no',
       'gp_code', 'block_code', 'block_name', 'sr_gp_combo', 'gp_name',
       'gp_reservation_status', 'elected_sarpanch_name', 'husband_spouse_name',
       'candidate_res_status', 'age', 'sex', 'education', 'address',
       'gp_name_fin', 'gp_res_status_fin', 'candidate_res_status_fin',
       'cand_sex_fin', 'age_fin', 'educ_fin'],
      dtype='object')

In [23]:
up_2010.to_csv("up_gp_sarpanch_2010_fixed.csv", index = False)